# Tutorial


## Traditional example
In this tutorial, we'll simulate a simple 2D localization problem as per the figure below. We'll assume that we have a robot following a typical bicycle model, which has access to noisy measurements of its forward velocity $v$ and angular velocity $\omega$. In addition, this robot will have a time-of-flight sensor that gives it range measurements to a few known landmarks in the environment. 


The first step is to define the state of the robot. We'll start with a more traditional approach and define the state of the robot to be a vector of the form $\mathbf{x} = [x, y, \theta]$, where $x$ and $y$ are the robot's position in the world and $\theta$ is its orientation. We'll also define the control inputs to be $\mathbf{u} = [v, \omega]$, the robot's forward and angular velocity. The process (motion) model of the robot is then given by:

$$
\begin{aligned}
    \dot{x} &= v \cos(\theta), \\
    \dot{y} &= v \sin(\theta), \\
    \dot{\theta} &= \omega.
\end{aligned}
$$

However, this is in continuous time, and we need to discretize it to use it in a filter. For now, we'll use the simple Euler discretization method, which gives us the following discrete-time process model:

$$
\begin{aligned}
    x_{k+1} &= x_k + v_k \cos(\theta_k) \Delta t, \\
    y_{k+1} &= y_k + v_k \sin(\theta_k) \Delta t, \\
    \theta_{k+1} &= \theta_k + \omega_k \Delta t.
\end{aligned}
$$

Lets now code up our state and process model using navlie's framework. Since our state is just a regular 3x1 vector, we can use a standard type from the built-in library: `navlie.lib.VectorState` 


In [ ]:
import numpy as np
import navlie as nav
from navlie.lib import VectorState

x = VectorState([0, 0, 0], stamp=0.0)
print(x)

The `VectorState` is a subclass of the abstract `State` class in `navlie`, which is one of the core primitive types. The value of the state is stored as a numpy array, and can be accessed directly through `x.value`. 

For the process model, we'll choose to define our own from scratch here. Process models in navlie *must* inherit from the abstract `navlie.ProcessModel` class and implement the `evaluate` and either the `input_covariance` or `covariance` methods.

In [ ]:
from navlie.lib import VectorInput

Q = np.eye(2) * 0.1**2 # Input noise covariance with 0.1 m/s of standard deviation

class BicycleModel(nav.ProcessModel):
    def evaluate(self, x: VectorState, u: nav.VectorInput, dt: float) -> VectorState:
        x_next = x.copy()
        x_next.value[0] += u.value[0] * dt * np.cos(x.value[2])
        x_next.value[1] += u.value[0] * dt * np.sin(x.value[2])
        x_next.value[2] += u.value[1] * dt
        return x_next

    def input_covariance(self, x: VectorState, u: VectorInput, dt: float) -> np.ndarray:
        return Q
    
process_model = BicycleModel() # instantiate it

The methods in `navlie` process models must always accept the arguments shown above: a `State` object, an `Input` object, and a float `dt`. The `evaluate` method must always return a valid (i.e. subclass) `State` object, and the `input_covariance` method must always return a square numpy array. There are more optional methods that can be implemented for performance reasons, but we will cover those later.

Moving on to the measurement model